# Primitives examples

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.1
qiskit-ibm-runtime~=0.40.1
```
</details>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.1980952380952381
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -1.4
>>> Standard errors for PUB 0: 1.7722556772505693
>>> Expectation values for PUB 1: 0.029296875
>>> Standard errors for PUB 1: 0.011935455626543277
>>> Expectation values for PUB 2: -1.3103448275862069
>>> Standard errors for PUB 2: 1.947569832100723


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.07279853  1.0169848   0.88185684  0.62129487  0.32078203 -0.02173798
  -0.34398879 -0.62834503 -0.85218744 -1.02197866 -1.06163578 -0.99348429
  -0.87069409 -0.61718228 -0.29963157 -0.00323132  0.28876258  0.61042589
   0.86011886  1.01639729  1.07602985]
 [ 0.00176254  0.3695456   0.66741464  0.88655694  1.03020384  1.07191726
   1.01463475  0.84278723  0.61277594  0.32988848 -0.01204401 -0.35015768
  -0.65830819 -0.87921303 -1.02756003 -1.06251705 -1.01845358 -0.85600627
  -0.61013213 -0.31050056  0.00381883]
 [-0.01733163 -0.32225082 -0.59661933 -0.86599399 -1.01199094 -1.05634817
  -1.00376576 -0.83544332 -0.65067052 -0.31784447 -0.00499386  0.31138183
   0.62041361  0.83368078  0.99260302  1.04812299  0.98555286  0.87275039
   0.62305741  0.30785675  0.01321904]
 [ 1.03490395  0.98437784  0.84190596  0.59779436  0.29316893 -0.03231321
  -0.33370732 -0.63539518 -0.85130617 -1.00846587 -1.05164806 -1.00141571
  -0.84190596 -0.59691309 -0.30991305 -0.0067

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.579185520361991
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.4346076458752515
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1001111010100100010000010110010010101101000111010000010011001101010100110100100101000010010000111010010010001000100010000000010', '0110110110011101001000111011010000010000101110000000111011100111100110111000010100011000100000100000111100000001010110000011001', '0001010100110001001100111110000001011011000011110101110011001011000111111100101010000000100011001010010010110001000100100100000', '1011001001100001000010101110110010100101000010011011001100101110100110100010100101000001000110011001110010101101000001001010010', '0110111001010100000010110011000100011001100101000010000000000100100000001000100010001100101000110001010100000101001100001000001', '1011111101010101110110000110001100000001100100000000100011011001001100011101000000101000001000011001110000100010000001100110010', '0010110000010000001100011001110110011111101100011001010111110010000000110100000000000010000001111000000101000000100100111011001', '11011000011010001101011100110100010010100001110100100000000

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0000101101000111000011001100100110011101000100000001101011100010000010100011010100110001000010011001000000000011010011000010111', '1100001110110000110000110110000110000011111010111100100101010100001011010100110000100001010011000010101010001110011000000100010', '0100100001001101011100010110101101101101011001010001010010010101100111000000110000011101000101111000010100000111000001010000000', '0100001101111110000000100100100000100001100101110000010000011100101111010110000000111000000010000001100100001100010000101001000', '0010101101110111110011111001000100111010011000101110100100101110000011100100010100011001000010000000000100000100001100110010110', '0110011000110001101111000001100100000110110100000010010001100011010010000100010110100000110000101100000000100000010011010010110', '0010101001100001111001111001000111011100110110000011001000000010100010101001101100011001101101110110000000000110010011000001111', '0010101011011010010000000110000010100110101001111

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['1001001000111011111011000101100000011101111110111100001010100100100100011010000100000011110100001000001101111101010101011011101', '0001001000001000100010000000110010000000111000011111010100100111001100101111111000001100000101100000001001100010110011011111000', '1110111000001101001001100101100100000011000011000110011100010110100001111111000111101110010101010100001111100110110101001000011', '0100111000110100100100000110110010101001000101110011111010111101101001111001001100011000101001001000001001110100111000010101001', '0000001010001001101000010111110011111000110000001111100010111101101011010111101010001110011000110100001101000001001110001001011', '1000011000001001100000000110110110010100100101110001011110010010010101100100101101001110001000101100011010000010011010001111101', '1011011001110001100000110101110111010110010010110110001111000100110010001100100001010001011010001110111001010010111001010101011', '00110110001110011101101100001

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0001100001111110000100111111101000001101010000101100110100010101001111100010010110111000101000100000010000101110000111101100000', '0111001111000011000011011010001101001000000000010110010100101100111011100010010000010100101000110000101010000000110100001000000', '0011101000101000001101101100111101101011011110101100110100000000101010000110000000011100000000111001010010101100000100011000001', '1011001101111010101001110100111100101010000100110111010000010011000001010110010100100101111001110101010001101001000001100001110', '0010000001100010110010101100101101010001010101000100000000011101000000110110000010101000010000000000000111101110000001110000001', '0101111111010110010000111110011101100001000010001110111011111010100110101000000100010000000010001100001101100001000011001101000', '0101010011000000011001011000000111001000110001000111000111000110001011000010010100000100001000101111111010001001001100110101000', '1010001110111001111001010011011111

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0110000000001100101100000001010100100001110110000011111110000000000000010100101010010101100011100000000000010100010000000000000', '1001000110000010001100110010101101011000000100101100010000110100010001100001010111100010010000000001001110001100001010000000101', '0110011011100000110111000010110011101101001010010000100101010100001101010000101010001100111001010001100001110110000100000000010', '0010110000001011001010011010010010100100101000010110000000110110000100110010010101001000101001010000011101110000010010001101100', '0001100011000000100110101110100101011100101000010010000000100101110000111111001101011011001010111011010100001011000000001010101', '0110110000101111101110000101110000110001101001000010001010000000101101100101001101010010110001001010101101001010101010001011100', '1100111010101010011000010101101110111001000001000101101010000100001001100001100001110011011010011000100000010101100000100001100', '1101000100101100000100011011001011

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Compare transpiler settings](/docs/tutorials/circuit-transpilation-settings) tutorial.
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>